In [1]:
import json
import pandas as pd
import glob
import datetime
from datetime import date, timedelta
import openpyxl
import pathlib
import numpy as np
from datetime import datetime
import os
import win32com.client
import webbrowser
import time
import random
from collections import OrderedDict
import csv
import re

In [2]:
def rename_csv_files(folder_path, title_column, resolve_time_column, sheet_name=None):
    files = os.listdir(folder_path)
    
    for file in files:
         if file.endswith(".csv") and "tcs_export" in file:
            file_path = os.path.join(folder_path, file)
            if sheet_name is None:
                df = pd.read_csv(file_path)
            else:
                df = pd.read_csv(file_path, sheet_name=sheet_name)
                
            first_title = df[title_column][0]
            
            df[resolve_time_column] = pd.to_datetime(df[resolve_time_column])
            
            start_date = df[resolve_time_column].min()
            end_date = df[resolve_time_column].max()
            
            first_title = re.sub(r'[\\/:"*?<>|]+', '', first_title)
            
            start_date_str = start_date.strftime("%Y-%m-%d")
            end_date_str = end_date.strftime("%Y-%m-%d")
            
            new_file_name = f"{first_title}_{start_date_str}_{end_date_str}.csv"
            new_file_path = os.path.join(folder_path, new_file_name)
            
            if os.path.exists(new_file_path):
                os.remove(new_file_path)
            
            os.rename(file_path, new_file_path)

dir_randomraw = 'Random rawdata'

rename_csv_files(dir_randomraw, 'title', '1_resolve_time')

In [3]:
#Queue
queue_list = pd.read_excel('Support_dashboard.xlsx',sheet_name='Queue List').values
queue_list = pd.DataFrame(queue_list)
headers = ['BUSINESS','COMPOUND','CATEGORY','MOD QUEUE ID','MOD QUEUE NAME',
           'QA QUEUE ID','QA QUEUE NAME','QUEUE GROUP','STATUS','Priority','Latency','AHT (secs)','Policy_type','Classify']
queue_list.columns = headers

#Policy Errors
policy_errors_list = pd.read_excel('Support_dashboard.xlsx',sheet_name='Policy Errors').values
policy_errors_list = [item for sublist in policy_errors_list for item in sublist]

#Data Alternation
data_full_alternation = pd.read_excel('linemanager_full.xlsx')
data_full_alternation = data_full_alternation[data_full_alternation['Role'].str.contains('Operator')]
data_full_alternation = data_full_alternation[['EffectDate','Email','FullName','LineManager','ProductionDate']]

In [4]:
from datetime import datetime

def convert_datetime(a):
    my_format="%Y-%m-%d %H:%M:%S"
    try:
        return datetime.strptime(str(a), "%Y-%m-%d %H:%M:%S").strftime(my_format)
    except:
        return datetime.strptime(str(a), "%m/%d/%Y %H:%M").strftime(my_format)
def convert_date(a):
    my_format="%Y-%m-%d"
    try:
        return datetime.strptime(str(a), "%Y-%m-%d %H:%M:%S").strftime(my_format)
    except:
        return datetime.strptime(str(a), "%m/%d/%Y %H:%M").strftime(my_format)    
    
def input_data(folder_path):
    file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

    dfs = []
    for file_path in file_paths:
        try:
            df = pd.read_csv(file_path, dtype={'task_id': object,'project_id':object,'object_id':object})
            df[['project_id','task_id','object_id']] = df[['project_id','task_id','object_id']].replace(to_replace=r'id=',value= '',regex=True).astype(str)
            df['1_resolve_time'] = df['1_resolve_time'].apply(convert_datetime)
            
            df['source_file'] = os.path.basename(file_path)

            dfs.append(df)
        except Exception as e:
            print(f"Error reading file {file_path}: {str(e)}")
  
    merged_df = pd.concat(dfs, axis=0, ignore_index=True)

    return merged_df


In [5]:
Rand_rawdata = input_data('Random rawdata')
Rand_rawdata = Rand_rawdata.drop_duplicates(subset='task_id')

In [6]:
Rand_rawdata = Rand_rawdata[['task_id', 'project_id', 'title', 'object_id','1_verifier','1_resolve_time','1_duration','1_verify_data']]

In [7]:
def filter_time( df,  from_date, to_date):
    Rand_rawdata['1_resolve_time'] = pd.to_datetime(Rand_rawdata['1_resolve_time'])
    Rand_rawdata_fiter = df[(df['1_resolve_time'].dt.strftime('%Y-%m-%d') >= from_date) & (df['1_resolve_time'].dt.strftime('%Y-%m-%d') <= to_date)]

    return Rand_rawdata_fiter,from_date,to_date

Rand_export,from_date,to_date = filter_time(Rand_rawdata, '2023-11-12', '2023-11-18')

In [8]:
Rand_export

,task_id,project_id,title,object_id,1_verifier,1_resolve_time,1_duration,1_verify_data
270712,7298434342366364162,7171514518084895234,VN LL Live Backup,7298201659228834561_1699259133,nguyen.npk@trans-cosmos.com.vn,2023-11-12 01:16:02,23,"{""real_assign_time"":1699726528.685,""real_resol..."
271237,7298441047066132994,7171514518084895234,VN LL Live Backup,7298343097758255873_1699282189,nguyen.npk@trans-cosmos.com.vn,2023-11-12 01:16:36,28,"{""real_assign_time"":1699726557.952,""real_resol..."
271456,7298443606312927746,7171514518084895234,VN LL Live Backup,7298367667625282306_1699284532,linh.pc@trans-cosmos.com.vn,2023-11-12 04:22:22,10,"{""real_assign_time"":1699737721.701,""real_resol..."
271519,7298444370380882434,7171514518084895234,VN LL Live Backup,7298102367072406274_1699225338,nguyen.npk@trans-cosmos.com.vn,2023-11-12 06:01:54,24,"{""real_assign_time"":1699743679.615,""real_resol..."
271600,7298445356193333762,7171514518084895234,VN LL Live Backup,7298245188672047880_1699256216,trang.nlx1@trans-cosmos.com.vn,2023-11-12 06:03:21,66,"{""real_assign_time"":1699743723.414,""real_resol..."
...,...,...,...,...,...,...,...,...
282650,7299537404195570178,7171514518084895234,VN LL Live Backup,7299115286928657158_1699495305,anh.nm@trans-cosmos.com.vn,2023-11-14 16:31:41,34,"{""real_assign_time"":1699954255.429,""real_resol..."
282651,7299537425427071489,7171514518084895234,VN LL Live Backup,7299415768637852434_1699529363,thai.ng@trans-cosmos.com.vn,2023-11-14 16:31:43,14,"{""real_assign_time"":1699954278.164,""real_resol..."
282652,7299537456712499714,7171514518084895234,VN LL Live Backup,7299347033675074311_1699514436,hao.hm@trans-cosmos.com.vn,2023-11-14 16:32:27,28,"{""real_assign_time"":1699954308.134,""real_resol..."
282653,7299537468720693761,7171514518084895234,VN LL Live Backup,7299152476635253510_1699499404,van.vtn@trans-cosmos.com.vn,2023-11-14 16:32:36,30,"{""real_assign_time"":1699954314.784,""real_resol..."


In [9]:
keymap = {"Not Approve":['"status":400','"audit_result":400','disapproval', 'disapproved','not approval','not approval','not approve' ,'not approval','disapprove','disapproved','(truncated)'] ,
    "Approve":[ '"status":200' ,'"audit_result":200', 'approval', 'approved','200' ,'200.0' ,'approve']}

def check_string(s):
    if isinstance(s, str):
        s_lower = s.lower()

        for approval_status, substrings in keymap.items():
            if any(substring in s_lower for substring in substrings):
                return approval_status

    return s

Rand_export['1st_result'] = Rand_export['1_verify_data'].apply(check_string)



C:\Users\v6210227\AppData\Local\Temp\ipykernel_19292\3927253285.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Rand_export['1st_result'] = Rand_export['1_verify_data'].apply(check_string)


In [10]:
Rand_export_test = Rand_export[['title', 'task_id', '1_verify_data', '1st_result']]
Rand_export_test = Rand_export_test.loc[Rand_export_test['1st_result'] == 'Not approve']
Rand_export_test = Rand_export_test.drop_duplicates(subset=['title'])

def get_first_not_approve(df):
    return df.iloc[0]

Rand_export_test = Rand_export_test.groupby('title').apply(get_first_not_approve)

In [11]:
Rand_export['1_verify_data'] = Rand_export['1_verify_data'].replace(['0013', '0012', '0005', '_'], ' ', regex=True)
Rand_export['1_verify_data'] = Rand_export['1_verify_data'].str.lower()

C:\Users\v6210227\AppData\Local\Temp\ipykernel_19292\3688987074.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Rand_export['1_verify_data'] = Rand_export['1_verify_data'].replace(['0013', '0012', '0005', '_'], ' ', regex=True)
C:\Users\v6210227\AppData\Local\Temp\ipykernel_19292\3688987074.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Rand_export['1_verify_data'] = Rand_export['1_verify_data'].str.lower()


In [12]:
Policy_errors = policy_errors_list.copy()
cols_to_check = ['1_verify_data']
cols_to_return = ['Mod Reason']
queue_policy_type = queue_list[['MOD QUEUE NAME','Policy_type']]
Rand_export = pd.merge(Rand_export,queue_policy_type, how='inner', left_on='title', right_on='MOD QUEUE NAME')
for col_idx, col in enumerate(cols_to_check):
    Rand_export[cols_to_return[col_idx]] = ''
    for i, row in Rand_export.iterrows():
        cell_value = row[col]
        policy_type = row['Policy_type']
        if isinstance(cell_value, str):
            if 'form info' in cell_value:
                form_info_index = cell_value.index('form info')
                cell_value = cell_value[form_info_index:]
                match_list_1 = [match for match in Policy_errors if match in cell_value]
                if match_list_1:
                    match_list = list(OrderedDict.fromkeys(match_list_1))
                    final_reason = ", ".join(match_list).strip()
                    Rand_export.at[i, cols_to_return[col_idx]] = final_reason
            elif 'reject reasons' in cell_value:
                form_info_index = cell_value.index('reject reasons')
                cell_value_new = cell_value[:form_info_index]
                match_list_1 = [match for match in Policy_errors if match in cell_value_new]
                if match_list_1:
                    match_list = list(OrderedDict.fromkeys(match_list_1))
                    final_reason = ", ".join(match_list).strip()
                    Rand_export.at[i, cols_to_return[col_idx]] = final_reason
            elif 'reject label' in cell_value:
                form_info_index = cell_value.index('reject label')
                cell_value = cell_value[form_info_index:]
                match_list_1 = [match for match in Policy_errors if match in cell_value]
                if match_list_1:
                    match_list = list(OrderedDict.fromkeys(match_list_1))
                    final_reason = ", ".join(match_list).strip()
                    Rand_export.at[i, cols_to_return[col_idx]] = final_reason
            else:
                if policy_type == 'Multi-choice':
                    match_list_1 = [match for match in Policy_errors if match in cell_value]
                    if match_list_1:
                        match_list = list(OrderedDict.fromkeys(match_list_1))
                        final_reason = ", ".join(match_list).strip()
                        Rand_export.at[i, cols_to_return[col_idx]] = final_reason
                elif policy_type == 'Single-choice':
                    match_list_1 = []
                    for match in Policy_errors:
                        if match in cell_value:
                            match_list_1.append(match)
                            break
                    if match_list_1:
                        final_reason = match_list_1[0]
                        Rand_export.at[i, cols_to_return[col_idx]] = final_reason
        else:
            continue

Rand_export['policy errors'] = Rand_export[['Mod Reason']].apply(lambda x: x[x != ''].loc[x[x != ''].last_valid_index()] if x[x != ''].last_valid_index() else '', axis=1)
Rand_export['policy errors'] = Rand_export['policy errors'].replace("_"," ",regex=True).replace("counterfeit abnorma price product","abnormal price")
Rand_export['policy errors'] = Rand_export['policy errors'].apply(lambda x: ", ".join(set(x.split(", "))))
Rand_export = Rand_export[['title','task_id','object_id','1_resolve_time','1_verifier','1_duration','1st_result','policy errors']]
Rand_export['Date'] = pd.to_datetime(Rand_export['1_resolve_time'].apply(convert_date))


In [13]:
Rand_export_rawdata = pd.merge(Rand_export,data_full_alternation,how='left',left_on=['Date','1_verifier'],right_on=['EffectDate','Email'])
Rand_export_rawdata.to_excel(f'all cases backup queue {from_date}{to_date}.xlsx')
Rand_export_rawdata = Rand_export_rawdata[Rand_export_rawdata['1_resolve_time'] >= Rand_export_rawdata['ProductionDate']]

In [14]:
Rand_export_rawdata = Rand_export_rawdata[['title', 'task_id', 'object_id', '1_resolve_time', '1_verifier', 'FullName', 'LineManager','1_duration', '1st_result', 'policy errors']]
def sample_or_return_all(x):
    if len(x) <= 10:
        return x
    else:
        return x.sample(n=10, replace=False, random_state=1)
Rand_export_for_QA = Rand_export_rawdata.groupby('1_verifier',as_index=False).apply(sample_or_return_all)
Rand_export_for_QA = Rand_export_for_QA[['title','task_id','object_id','1_resolve_time','1_verifier','FullName', 'LineManager','1_duration','1st_result','policy errors']]
Rand_export_for_QA = Rand_export_for_QA.reset_index(drop=True)

In [15]:
grouped_by_manager = Rand_export_for_QA.groupby('LineManager')
desired_rows_per_manager = 150
Rand_export_for_TLs = pd.DataFrame(columns=['title', 'task_id', 'object_id', '1_resolve_time', '1_verifier','FullName', 'LineManager', '1_duration', '1st_result', 'policy errors'])
for _, group in grouped_by_manager:
    sampled_group = group.sample(n=min(desired_rows_per_manager, len(group)), random_state=1)
    Rand_export_for_TLs = pd.concat([Rand_export_for_TLs, sampled_group])
Rand_export_for_TLs = Rand_export_for_TLs.reset_index(drop=True)
Rand_export_for_TLs['LineManager'].value_counts()

C:\Users\v6210227\AppData\Local\Temp\ipykernel_19292\143026491.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Rand_export_for_TLs = pd.concat([Rand_export_for_TLs, sampled_group])


LineManager
BÙI THỊ THANH THẢO        150
HÀ MINH ĐẠT               150
LẠI TRẦN ĐOÀN THUẬN       150
NGUYỄN THỊ TUYẾT NHUNG    150
NGUYỄN ĐỖ PHÚ YÊN         150
VÕ TẤN LỰC                150
NGUYỄN THỊ THU HẰNG        60
NGUYỄN HUY DANH            40
NGUYỄN QUANG ANH            2
NGUYỄN TRẦN ĐỨC THÀNH       2
LÊ TẤN THỊNH                1
Name: count, dtype: int64

In [16]:
# grouped_by_manager = Rand_export_for_QA.groupby('LineManager')
# Rand_export_for_TLs = pd.DataFrame(columns=['title', 'task_id', 'object_id', '1_resolve_time', '1_verifier', '1_duration', '1st_result', 'policy errors'])
# total_rows_to_select = 150
# for _, group in grouped_by_manager:
#     if len(group) > total_rows_to_select:
#         sampled_group = group.sample(n=total_rows_to_select, random_state=1)
#     else:
#         sampled_group = group
#     avg_rows_per_verifier = total_rows_to_select / len(sampled_group['1_verifier'].unique())    
#     temp_df = pd.DataFrame(columns=Rand_export_for_QA.columns)
#     for verifier, verifier_group in sampled_group.groupby('1_verifier'):
#         rows_for_verifier = min(int(avg_rows_per_verifier), len(verifier_group))
#         if len(verifier_group) > rows_for_verifier:
#             sampled_verifier_group = verifier_group.sample(n=rows_for_verifier, random_state=1)
#         else:
#             sampled_verifier_group = verifier_group
#         temp_df = pd.concat([temp_df, sampled_verifier_group])
#     Rand_export_for_TLs = pd.concat([Rand_export_for_TLs, temp_df])
# Rand_export_for_TLs = Rand_export_for_TLs.reset_index(drop=True)


In [17]:
test = Rand_export_for_QA.groupby(by=['title'],as_index=False).agg({'1_verifier':'count'})
print(sum(test['1_verifier']))
test

1261


,title,1_verifier
0,VN LL Live Backup,1261


In [18]:
Rand_export_for_QA.to_excel(f'Random case exported/QA Random case exported exported {from_date}{to_date}.xlsx', index=False)
Rand_export_for_TLs.to_excel(f'Random case exported/TL Random case exported exported {from_date}{to_date}.xlsx', index=False)